In [ ]:
import csv
import pandas as pd
import os
import opendatasets as od 
import pytz
from pytz import UTC
import  ciso8601
import influxdb_client
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.write.point import EPOCH


In [ ]:
od.download("https://www.kaggle.com/nphantawee/pump-sensor-data")

In [ ]:
# get the working directory 
os.getcwd()

In [ ]:
#set the working directory 
os.chdir('/mnt/c/Users/NicolaRonzoni/OneDrive - BitBang Srl/pump-sensor-data')

In [ ]:
os.getcwd()

In [ ]:
#load the dataset 
df = pd.read_csv('/mnt/c/Users/NicolaRonzoni/OneDrive - BitBang Srl/pump-sensor-data/sensor.csv',sep=',')

In [ ]:
df.info()

In [ ]:
#token api, ID bucket and name of the org from User Interface
my_token = 'ZrKUM72cuRCQe0l2ahfZQkb_Gmkax3jFm6fgM1CkA0-gzUlrezQmLegjFmCQiQzZcFt3rarBBwSi6r_XHIFTBw=='
my_org = "nronzoni@bitbang.com"
my_bucket = "6c07c0824c29c073"
url = "https://eu-central-1-1.aws.cloud2.influxdata.com"


In [ ]:
#making a connection, specify write options 
client = InfluxDBClient(url=url, token=my_token, org=my_org,debug=True)
write_client = client.write_api(write_options=WriteOptions(batch_size=1000,
flush_interval=10_000,
jitter_interval=2_000,
retry_interval=5_000))
 


In [ ]:
# specify an indicator in order to identify values of the series 
df.loc[:,'stamp']='Original'

In [ ]:
# select only columns of interest, set index of pandas df to datetime64. Add missing rows to jumps in the series. 
df1=df[["timestamp",'stamp','machine_status','sensor_42','sensor_43','sensor_44','sensor_45']]
df1['date'] = pd.to_datetime(df1['timestamp'])
df1 = df1.set_index('date')
df1.drop(['timestamp'], axis=1, inplace=True)
df1 = df1.asfreq('1Min')
df1.info()

In [ ]:
df1.head()

In [ ]:
#write the data specyfing the name of the dataframe and tag columns to identify series 
write_client.write(my_bucket, my_org, record=df1[:1440], data_frame_measurement_name="sensor-df", data_frame_tag_columns=['stamp'])

In [ ]:
#close the connection
write_client.__del__()
client.__del__()

In [ ]:
# from influxdb to pandas again 
from influxdb_client import InfluxDBClient

bucket = "water pump sensors"
#specify the connection and the query using flux language. 
client = InfluxDBClient(url=url, token=my_token, org=my_org, debug=False)
query = """option v = {timeRangeStart: 2018-04-01T00:00:00Z, timeRangeStop: 2018-04-02T00:00:00Z}

from(bucket: "water pump sensors")
    |> range(start: v.timeRangeStart, stop: v.timeRangeStop)
    |> filter(fn: (r) => r["_measurement"] == "sensor-df")
    |> filter(fn: (r) => r["_field"] == "sensor_44" or r["_field"] == "sensor_45")
    |> fill(usePrevious: true)

from(bucket: "water pump sensors")
    |> range(start: v.timeRangeStart, stop: v.timeRangeStop)
    |> filter(fn: (r) => r["_measurement"] == "sensor-df")
    |> filter(fn: (r) => r["_field"] == "sensor_44" or r["_field"] == "sensor_45")
    |> aggregateWindow(every: 20m, fn: stddev, createEmpty: true)
    |> yield(name: "stddev")"""
# dataframe saved in a list, therefore select the first entry
df2= client.query_api().query_data_frame(org=my_org, query=query)[0]





    
    

In [19]:
df2.head()

,result,table,_start,_stop,_time,_value,_field,_measurement,stamp
0,_result,0,2018-04-01 00:00:00+00:00,2018-04-02 00:00:00+00:00,2018-04-01 00:00:00+00:00,65.68287,sensor_45,sensor-df,Original
1,_result,0,2018-04-01 00:00:00+00:00,2018-04-02 00:00:00+00:00,2018-04-01 00:01:00+00:00,65.68287,sensor_45,sensor-df,Original
2,_result,0,2018-04-01 00:00:00+00:00,2018-04-02 00:00:00+00:00,2018-04-01 00:02:00+00:00,65.39352,sensor_45,sensor-df,Original
3,_result,0,2018-04-01 00:00:00+00:00,2018-04-02 00:00:00+00:00,2018-04-01 00:03:00+00:00,64.81481,sensor_45,sensor-df,Original
4,_result,0,2018-04-01 00:00:00+00:00,2018-04-02 00:00:00+00:00,2018-04-01 00:04:00+00:00,65.10416,sensor_45,sensor-df,Original
